In [ ]:
# update pip
!/opt/venv/bin/python -m pip install --upgrade pip

# module installs
!pip install pandas numpy tqdm plotly

Requirement already up-to-date: pip in /opt/venv/lib/python3.7/site-packages (20.2.4)


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
pd.options.plotting.backend = "plotly"
import plotly.io as pio
import plotly.graph_objects as plt
import plotly.express as px
pio.templates.default = "plotly_white"
from collections import defaultdict

/opt/venv/lib/python3.7/site-packages/tqdm/std.py:703: FutureWarning:

The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version



In [ ]:
NEUTRALSET = "./neutralTweets.csv"
NEUTRALROWS = 300000
DEPRESSEDSET = "./depressedTweets.csv"
DEPRESSEDROWS = 30000

In [ ]:
neutralSet = pd.read_csv(NEUTRALSET, nrows=NEUTRALROWS)
depressedSet = pd.read_csv(DEPRESSEDSET, nrows=DEPRESSEDROWS)

In [ ]:
neutralSet['target'] = 0

In [ ]:
depressedSet['target'] = 1

In [ ]:
mixedFinal = pd.concat([neutralSet, depressedSet], ignore_index=True)

In [ ]:
mixedFinal

,tweet,target
0,is so sad for my APL frie...,0
1,I missed the New Moon trail...,0
2,omg its already 7:30 :O,0
3,.. Omgaga. Im sooo im gunna CRy. I'...,0
4,i think mi bf is cheating on me!!! ...,0
...,...,...
329995,Feel like a depression cloud is forming above....,1
329996,"We have to stop talking about ""Mental Illness""...",1
329997,@tal7291 @DanLairdMD I disagree. If one has fi...,1
329998,J'vais tomber en dépression twitter.com/ojwess...,1


In [ ]:
# Expand Contractions
import re
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [ ]:
def trashmaster2000(str_argument):
    str_argument = expandContractions(str(str_argument))
    cleaned_str=re.sub('[^a-z\s]+',' ',str_argument,flags=re.IGNORECASE)
    cleaned_str=re.sub('(\s+)',' ',cleaned_str)
    cleaned_str=cleaned_str.lower()
    return cleaned_str

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

#making test train splits and cleanup
X_train, X_test, y_train, y_test = train_test_split(mixedFinal['tweet'].progress_apply(trashmaster2000), mixedFinal['target'], test_size=0.05)

100%|██████████| 330000/330000 [00:10<00:00, 30997.02it/s]


In [ ]:
X_train[:20]

18573      sunnystokes uh i was not really expecting a r...
166933      gabem hahaha yea i do i miss fighting with you 
164815     freshalicious uh fer sure i want some double ...
323570    depression i thought i would write something f...
56309      bbricke i used to own a comic book store so y...
312166    freshman year i only listened to elliott smith...
1318         i cannot hear a damn word that tbs is singing 
307805    goku has many forms in the hit anime dragon ba...
152129     erin im anita nice to kno you i find you bcz ...
320406     soliclarity someone wrote me a novel on stan ...
91346      chold i want to so badly i cannot get off wor...
308905     kojo bankz you dey this country you say you g...
173119     gloriavelez yep our lakers have this great ch...
217385             jc have a good morning baby i love you x
126815     djpsar fuck why do i still got exams and do i...
242097     katanasquirrel it does that sometimes after y...
285646     lmpotter my dad s memorial is

In [ ]:
y_train[:20]

18573     0
166933    0
164815    0
323570    1
56309     0
312166    1
1318      0
307805    1
152129    0
320406    1
91346     0
308905    1
173119    0
217385    0
126815    0
242097    0
285646    0
304222    1
327395    1
69147     0
Name: target, dtype: int64

In [ ]:
# encoding to bag of words
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

#encode test set

X_test_counts = count_vect.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# load and train the model
logistic_reg = LogisticRegression()

logistic_reg.fit(X_train_counts,y_train)

LogisticRegression()

In [ ]:
print("The train set has {} and test set has {}".format(len(X_train),len(X_test)))

The train set has 313500 and test set has 16500


In [ ]:
# get predcitions for test set
y_pred= logistic_reg.predict(X_test_counts)

In [ ]:
y_pred

array([0, 0, 0, ..., 0, 0, 1])

In [ ]:
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score
accuracy_score(y_test, y_pred)

1.0

In [ ]:
fpr, tpr, thresholds = roc_curve(y_pred, y_test)
roc_auc = auc(fpr, tpr)

In [ ]:
fig = plt.Figure()

fig.add_trace(plt.Scatter(
    x=fpr,
    y=tpr,
    name="ROC curve (area = {0:.4f})".format(roc_auc),
    fill = 'tozeroy'
))

fig.add_trace(plt.Scatter(
    x=[0,1],
    y=[0,1],
    name="Line",
    showlegend=False,
    line=dict(dash='dashdot')
))

fig.update_layout(
    title="ROC Curve",
    xaxis_title="FPR",
    yaxis_title="TPR",
    legend_title="",
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1))
fig.show()

In [ ]:
roc_auc_score(y_test, y_pred)

1.0

In [ ]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     15024
           1       1.00      1.00      1.00      1476

    accuracy                           1.00     16500
   macro avg       1.00      1.00      1.00     16500
weighted avg       1.00      1.00      1.00     16500

